In [ ]:
import sys
sys.path.insert(0, "../Src/")
import generateVoices as gv
import evaluateVoices as ev
import pandas as pd
import os
from os.path import exists
import json

%autosave 5

In [ ]:
# df = pd.read_csv("../Data/test_data.csv")
# df

In [ ]:
def getBestModelName():
    wordErrorRateResults = pd.read_csv("../Data/wordErrorRateResults.csv")
    bestModel = wordErrorRateResults.sort_values(by="averageWER").iloc[0]
    return bestModel["model"]

In [ ]:
def getSourceVoice(speaker,speakerDf,datasetName="train"):
    if datasetName == "train":
        sourceVoiceFile = f"../Data/concatenatedInput/{speaker}.wav"
    else:
        sourceVoiceFile = f"../Data/concatenatedTestInputs/{speaker}.wav"
    if exists(sourceVoiceFile):
        return sourceVoiceFile
    gv.concatenateAudio(speaker, speakerDf,datasetName)
    if exists(sourceVoiceFile):
        return sourceVoiceFile
    else:
        print(f"Source voice file for {speaker} not found after concatenation.")
        return None

In [ ]:
def getFADJson(datasetName):
    fadJson = []
    if exists(f"../Data/fad_{datasetName}.json"):
        with open(f"../Data/fad_{datasetName}.json", "r") as f:
            fadJson = json.load(f)
    return fadJson

In [ ]:
def saveFADJson(datasetName, fadJson):
    with open(f"../Data/fad_{datasetName}.json", "w") as f:
        json.dump(fadJson, f)

In [ ]:
def createFakeAudio(speakerDf, sourceVoice, modelName, datasetName):
    fadJson = getFADJson(datasetName)
    textFiles = speakerDf[speakerDf['path_from_data_dir'].str.contains('.TXT', na=False)]
    textFileNames = textFiles['path_from_data_dir'].tolist()

    for textFileName in textFileNames:
        realFile = f"Data/data/{datasetName}/{textFileName.replace('.TXT', '.WAV.wav')}"
        fakeAudioFile = f"../Data/fakeAudio/{datasetName}/{textFileName.replace('.TXT', '.wav')}"
        if exists(fakeAudioFile):
            print(f"Fake audio file {fakeAudioFile} already exists.")
            continue
        os.makedirs(os.path.dirname(fakeAudioFile), exist_ok=True)
        model = gv.generateTTS(modelName)
        sentence = gv.readSentenceFromFile(textFileName)
        gv.generateAndNormalizeAudio(model, sentence, sourceVoice, fakeAudioFile)
        fadJson.append({
            "file": fakeAudioFile,
            "text": sentence,
            "isFake": True,
        })
        fadJson.append({
            "file": realFile,
            "text": sentence,
            "isFake": False,
        })
        saveFADJson(datasetName,fadJson)



In [ ]:
def prepareFADData(modelName,datasetName):
    print(f"Creating voices for {modelName} on {datasetName} dataset")
    
    df = gv.readCsv(datasetName)
    speakers = gv.getSpeakers(df)
    for speaker in speakers:
        speakerDf = gv.getFilesBySpeaker(df, speaker)
        sourceVoice = getSourceVoice(speaker, speakerDf,datasetName)
        if sourceVoice is None:
            print(f"Skipping {speaker} due to missing source voice.")
            continue
        createFakeAudio(speakerDf, sourceVoice, modelName, datasetName)
    voiceDF = pd.DataFrame(getFADJson(datasetName))
    voiceDF.to_csv(f"../Data/fad_{datasetName}.csv", index=False)
    return voiceDF



In [ ]:
def main():
    bestModelName = getBestModelName()
    trainDF = prepareFADData(bestModelName, 'train')
    testDF =  prepareFADData(bestModelName, 'test')
    display(trainDF.head())
    display(testDF.head())
    print('done')

In [ ]:
if __name__ == '__main__':
    main()